In [1]:
import pandas as pd
from googleapiclient.discovery import build
import scrapetube

In [3]:
api_key = 'api-key'
youtube = build('youtube', 'v3', developerKey=api_key)

search_response = youtube.search().list(
    q = '삼프로tv',
    order = 'relevance',
    part = 'snippet',
    maxResults = 10
    ).execute()

channel_id = search_response['items'][0]['snippet']['channelId']

In [4]:
channel_id

'UChlv4GSd7OQl3js-jkLOnFA'

In [5]:
videos = scrapetube.get_channel(channel_id)

video_ids = []

for video in videos:
    video_ids.append(video['videoId'])

In [6]:
len(video_ids)

8414

In [7]:
video_infos = []

for i in range(100):
    start = i * 50 
    end = (i + 1) * 50
    video_request = youtube.videos().list(
        part="snippet",
        id=','.join(video_ids[start:end]))

    video_response = video_request.execute()

    for item in video_response['items']:
        title = item['snippet']['title']
        if ('글로벌 이슈체크' in title) or ('글로벌 마켓브리핑' in title) or ('직장인 vlog' in title):
            continue
        video_infos.append([item['snippet']['title'], item['snippet']['publishedAt'], item['id']])
        
df = pd.DataFrame(video_infos, columns=['title', 'date', 'id'])
df.head()

,title,date,id
0,[오늘아침 라이브] '여의도 닥터둠' 강영현이 보는 진바닥 시점_22.07.08_박...,2022-07-07T21:19:41Z,_eWj19Q9Ua8
1,"주식같은 채권, 영구채의 세계/잡혀가는 물가, 올해 시장은 선방한다!/변동금리 대출...",2022-07-07T14:23:50Z,C-IEaZmzPRY
2,[글로벌 개장시황] 내년 금리인하? 의사록의 답변은? f. 신한금융투자 최진석 수...,2022-07-07T14:04:18Z,tFWH1xFecb8
3,[심층인터뷰] 경기 침체? 정말 팬데믹 때보다 나빠질까? f. 메리츠증권 김웅 부장,2022-07-07T11:12:33Z,cgPdmP8xbo4
4,"[증시셔터맨] EU 택소노미에 원전 포함, 우리 기업에 호재일까? f. IBK투자증...",2022-07-07T11:12:05Z,zXIOudrelrU


In [8]:
len(df)

4504

In [9]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [10]:
df

,title,date,id
0,[오늘아침 라이브] '여의도 닥터둠' 강영현이 보는 진바닥 시점_22.07.08_박...,2022-07-07 21:19:41+00:00,_eWj19Q9Ua8
1,"주식같은 채권, 영구채의 세계/잡혀가는 물가, 올해 시장은 선방한다!/변동금리 대출...",2022-07-07 14:23:50+00:00,C-IEaZmzPRY
2,[글로벌 개장시황] 내년 금리인하? 의사록의 답변은? f. 신한금융투자 최진석 수...,2022-07-07 14:04:18+00:00,tFWH1xFecb8
3,[심층인터뷰] 경기 침체? 정말 팬데믹 때보다 나빠질까? f. 메리츠증권 김웅 부장,2022-07-07 11:12:33+00:00,cgPdmP8xbo4
4,"[증시셔터맨] EU 택소노미에 원전 포함, 우리 기업에 호재일까? f. IBK투자증...",2022-07-07 11:12:05+00:00,zXIOudrelrU
...,...,...,...
4499,이다솔 이효석 이한영이 알려주는 21년 명확한 투자 방향 [신과 함께 #162-2],2021-01-24 00:00:02+00:00,4pg1MiYiP9Y
4500,미스터 마켓 저자들이 2021년을 준비하는 법 [신과함께 #162-1],2021-01-23 23:00:08+00:00,hi2KVMIihoM
4501,하루가 너무 짧아서 고민? 강한 성공욕에 투잡을 넘어 포잡까지! (염승환 부장 &...,2021-01-23 01:56:07+00:00,-_Ftf2NaOiw
4502,"인공지능 투자의 시대, 로보어드바이저의 A to Z [삼프로TV X 파운트]",2021-01-23 01:00:03+00:00,RsdLug0rLr4


In [11]:
df = df.set_index('date')

In [12]:
df_21 = df[(pd.DatetimeIndex(df.index).year == 2021) & (pd.DatetimeIndex(df.index).month >= 6)]
df_22 = df[(pd.DatetimeIndex(df.index).year == 2022) & (pd.DatetimeIndex(df.index).month <= 6)]

In [13]:
df_1y = pd.concat([df_22, df_21])
df_1y

,title,id
date,,
2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk
2022-06-30 14:04:45+00:00,[글로벌 개장시황] 유가 하락 전망하는 3가지 이유 f. 미국주식에미치다 장우석 부사장,7SsI_15R4z4
2022-06-30 12:05:00+00:00,[심층인터뷰] 하락장에서도 버티는 제약·바이오의 비밀 f. 현대차증권 엄민용 책임연구원,lOkhqyvmkC4
2022-06-30 12:04:47+00:00,"[증시셔터맨] 상승하는 편의점과 마트, 더 갈 수 있나? f. IBK투자증권 박근형 부장",cOwbFklaWJs
2022-06-30 02:14:32+00:00,[심층인터뷰] 하락장은 이렇게 진행됩니다 f.김한진 이코노미스트,I-TeO8bJKfQ
...,...,...
2021-06-01 01:41:53+00:00,[증시 염탐정] 기관·외국인 매수 출발.. 종가에는 어떨까? f.이베스트투자증권 염...,K16i24lDTGw
2021-06-01 01:26:30+00:00,"[심층 인터뷰] 미국에서의 현지 생산.. 점유율과 실적, 동반 상승한다? f.이베스...",UOeGThyaVNA
2021-06-01 01:11:52+00:00,[제길선생의 미국 돋보기] 테이퍼링 분수령 될 6월.. 시장은 박스권을 유지할까? ...,524lLbP7z1k


In [43]:
# comments = []

# for video_id in df_1y['id']:
#     api_obj = build('youtube', 'v3', developerKey=api_key)
#     response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=100).execute()
 
#     while response:
#         for item in response['items']:
#             comment = item['snippet']['topLevelComment']['snippet']
#             comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])

# #             if item['snippet']['totalReplyCount'] > 0:
# #                 for reply_item in item['replies']['comments']:
# #                     reply = reply_item['snippet']
# #                     comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])
 
#         if 'nextPageToken' in response:
#             response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
#         else:
#             break

# df = pd.DataFrame(comments)
# df.to_excel('sampro_06.xlsx', header=['comment', 'author', 'date', 'num_likes'], index=None)

In [61]:
comments_21 = []

for video_id in df_21['id']:
    api_obj = build('youtube', 'v3', developerKey=api_key)
    response = api_obj.commentThreads().list(part='snippet', videoId=video_id, maxResults=100).execute()
 
    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments_21.append([video_id, comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])

        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
        else:
            break

df0 = pd.DataFrame(comments_21, columns=['id', 'comment', 'author', 'comment_date', 'num_likes'])
print(len(df0))
df0.head()

178458


,id,comment,author,comment_date,num_likes
0,41txImmP8BM,새해복많이받으시고 성투합시다우리!,조바이든,2022-01-02T22:52:47Z,1
1,41txImmP8BM,삼프로 tv 새해 복많이 받으세요~~<br>BTS와 이찬원 초청하면 좋겠습니다,강수자,2022-01-02T22:16:09Z,0
2,41txImmP8BM,죄송한데 노래 좀 안나왔으면 ~~~,이은정,2022-01-02T15:59:06Z,1
3,41txImmP8BM,세 분 표정이 넘 따뜻해요~^^ 아빠 미소로 따뜻한 위로 되셨을것 같아요♡,지혜로운자,2022-01-02T11:12:59Z,1
4,41txImmP8BM,"예~ㄱ니, 멋지네요!",김형숙,2022-01-02T07:57:12Z,1


In [63]:
df_21.head()

,video_date,title,id
0,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM
1,2021-12-31 00:00:15+00:00,삼프로PD 맘대로 뽑은 2021 삼프로TV 어워드!,q1utzJ_aoBM
2,2021-12-30 23:00:04+00:00,이것이 라이브다! 삼프로PD 픽 2021 라이브 모먼트!,A6XOg5By9Fc
3,2021-12-30 14:13:41+00:00,[글로벌 개장시황] 장엔젤과 미리보는 2022년 1월 전망 f. 미국주식에미치다 장...,xB3rQPP1VgA
4,2021-12-30 14:10:22+00:00,"호피한 맥주맛이군. 형형색색 수제맥주 시장/패하지 않는 투자, 2022년은 이렇게/...",2kZsiasPSn4


In [66]:
sam_21 = pd.merge(df_21, df0, on='id', how='outer')
sam_21.head()

,video_date,title,id,comment,author,comment_date,num_likes
0,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,새해복많이받으시고 성투합시다우리!,조바이든,2022-01-02T22:52:47Z,1
1,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,삼프로 tv 새해 복많이 받으세요~~<br>BTS와 이찬원 초청하면 좋겠습니다,강수자,2022-01-02T22:16:09Z,0
2,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,죄송한데 노래 좀 안나왔으면 ~~~,이은정,2022-01-02T15:59:06Z,1
3,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,세 분 표정이 넘 따뜻해요~^^ 아빠 미소로 따뜻한 위로 되셨을것 같아요♡,지혜로운자,2022-01-02T11:12:59Z,1
4,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,"예~ㄱ니, 멋지네요!",김형숙,2022-01-02T07:57:12Z,1


In [34]:
sam_21.to_csv('sampro_21_2.csv', header=['video_date', 'title', 'id', 'comment', 'author', 'comment_date', 'num_likes'], index=None)

In [35]:
sam_21 = pd.read_csv('sampro_21_2.csv')
sam_21.head()

,video_date,title,id,comment,author,comment_date,num_likes
0,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,새해복많이받으시고 성투합시다우리!,조바이든,2022-01-02T22:52:47Z,1
1,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,삼프로 tv 새해 복많이 받으세요~~<br>BTS와 이찬원 초청하면 좋겠습니다,강수자,2022-01-02T22:16:09Z,0
2,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,죄송한데 노래 좀 안나왔으면 ~~~,이은정,2022-01-02T15:59:06Z,1
3,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,세 분 표정이 넘 따뜻해요~^^ 아빠 미소로 따뜻한 위로 되셨을것 같아요♡,지혜로운자,2022-01-02T11:12:59Z,1
4,2021-12-31 23:00:03+00:00,[고민상담] 애널리스트 목표가 상향시 따라사기 괜찮을까요? (박병창 부장 & 최예근...,41txImmP8BM,"예~ㄱ니, 멋지네요!",김형숙,2022-01-02T07:57:12Z,1


In [16]:
comments_22 = []

for video_id in df_22['id']:
    api_obj = build('youtube', 'v3', developerKey=api_key)
    response = api_obj.commentThreads().list(part='snippet', videoId=video_id, maxResults=100).execute()
 
    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments_22.append([video_id, comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
 
        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
        else:
            break

df1 = pd.DataFrame(comments_22, columns=['id', 'comment', 'author', 'comment_date', 'num_likes'])
print(len(df1))
df1.head()

96040


,id,comment,author,comment_date,num_likes
0,iHXeZ5v15dk,안녕하세요! 삼프로TV입니다.\r<br>여러분의 응원과 사랑으로 삼프로TV가 구독자...,삼프로TV_경제의신과함께,2022-06-30T13:54:12Z,0
1,iHXeZ5v15dk,탁프로님 분석 명확하고 구체적이라서 도움 많이 받습니다,상상해보자,2022-07-07T02:52:59Z,0
2,iHXeZ5v15dk,김탁님 분석 최고.,자전거55,2022-07-04T08:00:00Z,0
3,iHXeZ5v15dk,산업구조가 일본에서 한국 그 다음 중국 쪽으로 흘러가는 것. 한국이 살 길은 기술...,꿈이야!,2022-06-30T23:18:01Z,0
4,iHXeZ5v15dk,"누리호 얘기를 듣는데, 사회자분들의 코멘트가 너무 방해된다는 생각이 들정도로 매끄럽...",롱타이,2022-06-30T22:33:27Z,1


In [20]:
df_22 = df_22.reset_index()
df_22.head()

,date,title,id
0,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk
1,2022-06-30 14:04:45+00:00,[글로벌 개장시황] 유가 하락 전망하는 3가지 이유 f. 미국주식에미치다 장우석 부사장,7SsI_15R4z4
2,2022-06-30 12:05:00+00:00,[심층인터뷰] 하락장에서도 버티는 제약·바이오의 비밀 f. 현대차증권 엄민용 책임연구원,lOkhqyvmkC4
3,2022-06-30 12:04:47+00:00,"[증시셔터맨] 상승하는 편의점과 마트, 더 갈 수 있나? f. IBK투자증권 박근형 부장",cOwbFklaWJs
4,2022-06-30 02:14:32+00:00,[심층인터뷰] 하락장은 이렇게 진행됩니다 f.김한진 이코노미스트,I-TeO8bJKfQ


In [30]:
sam_22 = pd.merge(df_22, df1, on='id', how='outer')
sam_22.rename(columns={'date': 'video_date'}, inplace=True)
sam_22.head()

,video_date,title,id,comment,author,comment_date,num_likes
0,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,안녕하세요! 삼프로TV입니다.\r<br>여러분의 응원과 사랑으로 삼프로TV가 구독자...,삼프로TV_경제의신과함께,2022-06-30T13:54:12Z,0
1,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,탁프로님 분석 명확하고 구체적이라서 도움 많이 받습니다,상상해보자,2022-07-07T02:52:59Z,0
2,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,김탁님 분석 최고.,자전거55,2022-07-04T08:00:00Z,0
3,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,산업구조가 일본에서 한국 그 다음 중국 쪽으로 흘러가는 것. 한국이 살 길은 기술...,꿈이야!,2022-06-30T23:18:01Z,0
4,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,"누리호 얘기를 듣는데, 사회자분들의 코멘트가 너무 방해된다는 생각이 들정도로 매끄럽...",롱타이,2022-06-30T22:33:27Z,1


In [33]:
sam_22.to_csv('sampro_22_2.csv', header=['video_date', 'title', 'id', 'comment', 'author', 'comment_date', 'num_likes'], index=None)

In [63]:
sam_1y = pd.concat([sam_22, sam_21])
sam_1y.rename(columns={'date': 'video_date', 'title': 'video_title'}, inplace=True)
sam_1y.head()

,video_date,video_title,id,comment,author,comment_date,num_likes
0,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,안녕하세요! 삼프로TV입니다.\r<br>여러분의 응원과 사랑으로 삼프로TV가 구독자...,삼프로TV_경제의신과함께,2022-06-30T13:54:12Z,0
1,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,탁프로님 분석 명확하고 구체적이라서 도움 많이 받습니다,상상해보자,2022-07-07T02:52:59Z,0
2,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,김탁님 분석 최고.,자전거55,2022-07-04T08:00:00Z,0
3,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,산업구조가 일본에서 한국 그 다음 중국 쪽으로 흘러가는 것. 한국이 살 길은 기술...,꿈이야!,2022-06-30T23:18:01Z,0
4,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,"누리호 얘기를 듣는데, 사회자분들의 코멘트가 너무 방해된다는 생각이 들정도로 매끄럽...",롱타이,2022-06-30T22:33:27Z,1


In [64]:
sam_1y.isnull().sum()  

video_date       0
video_title      0
id               0
comment          0
author          12
comment_date     0
num_likes        0
dtype: int64

In [65]:
sam_1y[sam_1y['author'].isnull()]  # 왜지?

,video_date,video_title,id,comment,author,comment_date,num_likes
5955,2021-12-24 23:00:34+00:00,[대선 특집] 삼프로가 묻고 이재명 후보가 답하다,y6DlTb3t8Bo,이재명 후보님 ㅠ 진심으로 당신은 자격이 충분합니다 ㅠ 능력잇는 대통령이 되어주세요,NaN,2022-01-06T15:56:27Z,92
18483,2021-12-24 23:00:34+00:00,[대선 특집] 삼프로가 묻고 이재명 후보가 답하다,y6DlTb3t8Bo,나라 구한 채널 구경하러 왔습니다. 구경해 보니 다른 후보는 2배속이 심신 건강에 ...,NaN,2021-12-29T05:49:49Z,16
21083,2021-12-24 23:00:34+00:00,[대선 특집] 삼프로가 묻고 이재명 후보가 답하다,y6DlTb3t8Bo,각각 하나만 보고 끄면 안됨 두개를 같이 봐야 ㄹㅇ 진가를 느낄수 있음 ;;,NaN,2021-12-28T09:56:52Z,12
24430,2021-12-24 23:00:34+00:00,[대선 특집] 삼프로가 묻고 이재명 후보가 답하다,y6DlTb3t8Bo,소문듣고 찾아왔다가 집중하며 봤네요<br>경제 전반에 걸친 해박함에 <br>박수를 ...,NaN,2021-12-27T10:27:47Z,1144
55947,2021-12-24 23:00:06+00:00,[대선 특집] 삼프로가 묻고 윤석열 후보가 답하다,DFzn7PP7--0,"방송 잘듣고, 보고있습니다.<br>안철수 후보영상도 올려주셔요",NaN,2021-12-31T10:21:45Z,0
69890,2021-12-24 23:00:06+00:00,[대선 특집] 삼프로가 묻고 윤석열 후보가 답하다,DFzn7PP7--0,"나라를 구했다, 절감하며 후속편 기대합니다.<br>물론 누군가는 응하지않겠지만..",NaN,2021-12-27T06:27:50Z,57
95120,2021-12-22 07:34:23+00:00,🎄크리스마스 선물 받아가세요🎄,VeI73aJ7qAQ,항상 잘 듣고 있습니다! <br>건강하시고 행복하세요~~^^,NaN,2021-12-25T08:31:59Z,0
97715,2021-12-22 07:34:23+00:00,🎄크리스마스 선물 받아가세요🎄,VeI73aJ7qAQ,메리 크리스마스! 올한해 삼프로TV 덕분에 투자에 많은 도움이 되었습니다. 앞으로도...,NaN,2021-12-23T22:57:13Z,0
102627,2021-12-15 14:19:23+00:00,"모빌리티 기업은 왜 주차장에 집착할까/강.박.솔의 연말 투자 전략, 이 영상 하나로...",Nmct5Q2hGPg,"<a href=""https://www.youtube.com/watch?v=Nmct5...",NaN,2021-12-16T05:33:16Z,0
111665,2021-11-19 13:16:44+00:00,"[백브리핑 라이브] 세금폭탄 주의! 해외주식 양도세, 증권사마다 계산법이 다르다? ...",pfsG78XqDiw,연말 돌아와서 해외주식 세금 궁금했었는데 확실히 알겠네요 정말 유익한 정보입니다...,NaN,2021-11-20T04:24:43Z,5


In [102]:
sam_1y['video_date'] = pd.to_datetime(sam_1y['video_date'], utc=True)
sam_1y['comment_date'] = pd.to_datetime(sam_1y['comment_date'], utc=True)

sam_1y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274498 entries, 0 to 178457
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   video_date    274498 non-null  datetime64[ns, UTC]
 1   video_title   274498 non-null  object             
 2   id            274498 non-null  object             
 3   comment       274498 non-null  object             
 4   author        274486 non-null  object             
 5   comment_date  274498 non-null  datetime64[ns, UTC]
 6   num_likes     274498 non-null  int64              
 7   date_diff     274498 non-null  object             
 8   market_hour   274498 non-null  bool               
 9   5             2 non-null       object             
dtypes: bool(1), datetime64[ns, UTC](2), int64(1), object(6)
memory usage: 29.3+ MB


In [67]:
sam_1y.head()

,video_date,video_title,id,comment,author,comment_date,num_likes
0,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,안녕하세요! 삼프로TV입니다.\r<br>여러분의 응원과 사랑으로 삼프로TV가 구독자...,삼프로TV_경제의신과함께,2022-06-30 13:54:12+00:00,0
1,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,탁프로님 분석 명확하고 구체적이라서 도움 많이 받습니다,상상해보자,2022-07-07 02:52:59+00:00,0
2,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,김탁님 분석 최고.,자전거55,2022-07-04 08:00:00+00:00,0
3,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,산업구조가 일본에서 한국 그 다음 중국 쪽으로 흘러가는 것. 한국이 살 길은 기술...,꿈이야!,2022-06-30 23:18:01+00:00,0
4,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,"누리호 얘기를 듣는데, 사회자분들의 코멘트가 너무 방해된다는 생각이 들정도로 매끄럽...",롱타이,2022-06-30 22:33:27+00:00,1


In [69]:
sam_1y['date_diff'] = 0

for idx in sam_1y.index:
    v = sam_1y.loc[idx, 'video_date']
    c = sam_1y.loc[idx, 'comment_date']
    diff = c - v
    sam_1y.loc[idx, 'date_diff'] = diff

print(sam_1y.info())
sam_1y.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274498 entries, 0 to 178457
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   video_date    274498 non-null  datetime64[ns, UTC]
 1   video_title   274498 non-null  object             
 2   id            274498 non-null  object             
 3   comment       274498 non-null  object             
 4   author        274486 non-null  object             
 5   comment_date  274498 non-null  datetime64[ns, UTC]
 6   num_likes     274498 non-null  int64              
 7   date_diff     274498 non-null  object             
dtypes: datetime64[ns, UTC](2), int64(1), object(5)
memory usage: 26.9+ MB
None


,video_date,video_title,id,comment,author,comment_date,num_likes,date_diff
0,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,안녕하세요! 삼프로TV입니다.\r<br>여러분의 응원과 사랑으로 삼프로TV가 구독자...,삼프로TV_경제의신과함께,2022-06-30 13:54:12+00:00,0,-1 days +23:42:07
1,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,탁프로님 분석 명확하고 구체적이라서 도움 많이 받습니다,상상해보자,2022-07-07 02:52:59+00:00,0,6 days 12:40:54
2,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,김탁님 분석 최고.,자전거55,2022-07-04 08:00:00+00:00,0,3 days 17:47:55
3,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,산업구조가 일본에서 한국 그 다음 중국 쪽으로 흘러가는 것. 한국이 살 길은 기술...,꿈이야!,2022-06-30 23:18:01+00:00,0,0 days 09:05:56
4,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,"누리호 얘기를 듣는데, 사회자분들의 코멘트가 너무 방해된다는 생각이 들정도로 매끄럽...",롱타이,2022-06-30 22:33:27+00:00,1,0 days 08:21:22


In [127]:
# 장 시간 
sam_1y['market_hour'] = False
days = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']

for idx in sam_1y.index:
    h = sam_1y.iloc[idx, 5].hour
    m = sam_1y.iloc[idx, 5].minute
    w = days[sam_1y.iloc[idx, 5].weekday()]
    
    if w != '토요일' and w != '일요일':
        if (datetime.time(h, m) > datetime.time(9, 0)) and (datetime.time(h, m) < datetime.time(15, 30)):
            sam_1y.loc[idx, 'market_hour'] = True

In [130]:
sam_1y.columns

Index([  'video_date',  'video_title',           'id',      'comment',
             'author', 'comment_date',    'num_likes',    'date_diff',
        'market_hour',              5,   'markethour'],
      dtype='object')

In [132]:
samtv = sam_1y.drop([5, 'markethour'], axis=1)
samtv

,video_date,video_title,id,comment,author,comment_date,num_likes,date_diff,market_hour
0,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,안녕하세요! 삼프로TV입니다.\r<br>여러분의 응원과 사랑으로 삼프로TV가 구독자...,삼프로TV_경제의신과함께,2022-06-30 13:54:12+00:00,0,-1 days +23:42:07,True
1,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,탁프로님 분석 명확하고 구체적이라서 도움 많이 받습니다,상상해보자,2022-07-07 02:52:59+00:00,0,6 days 12:40:54,False
2,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,김탁님 분석 최고.,자전거55,2022-07-04 08:00:00+00:00,0,3 days 17:47:55,False
3,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,산업구조가 일본에서 한국 그 다음 중국 쪽으로 흘러가는 것. 한국이 살 길은 기술...,꿈이야!,2022-06-30 23:18:01+00:00,0,0 days 09:05:56,False
4,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,"누리호 얘기를 듣는데, 사회자분들의 코멘트가 너무 방해된다는 생각이 들정도로 매끄럽...",롱타이,2022-06-30 22:33:27+00:00,1,0 days 08:21:22,False
...,...,...,...,...,...,...,...,...,...
178453,2021-06-01 00:44:02+00:00,"국내 전기차 업계.. 다시 치고 올라가나?_오늘아침 page2_유지웅, 박병창, 염승환",Qn2zNAAn7qw,블랙핑크와 bts는 전세계 1위입니다,서프라이즈슈퍼,2021-06-01 00:53:22+00:00,0,0 days 00:09:20,False
178454,2021-06-01 00:44:02+00:00,"국내 전기차 업계.. 다시 치고 올라가나?_오늘아침 page2_유지웅, 박병창, 염승환",Qn2zNAAn7qw,잘나다니까. 그저 앵커나 모든 사람이 잘해서 잘나갈 수 밖에 없다고 알지요. 그것은...,조금성,2021-06-01 00:51:03+00:00,0,0 days 00:07:01,False
178455,2021-06-01 00:44:02+00:00,"국내 전기차 업계.. 다시 치고 올라가나?_오늘아침 page2_유지웅, 박병창, 염승환",Qn2zNAAn7qw,오늘도 좋은 내용 감사감사~,오리,2021-06-01 00:50:35+00:00,2,0 days 00:06:33,False
178456,2021-06-01 00:44:02+00:00,"국내 전기차 업계.. 다시 치고 올라가나?_오늘아침 page2_유지웅, 박병창, 염승환",Qn2zNAAn7qw,스팩관련주 너무 도박인가여?,Gu Park,2021-06-01 00:43:28+00:00,3,-1 days +23:59:26,False


In [133]:
samtv.to_csv('sampro_1y.csv', index=False)

In [134]:
pd.read_csv('sampro_1y.csv')

,video_date,video_title,id,comment,author,comment_date,num_likes,date_diff,market_hour
0,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,안녕하세요! 삼프로TV입니다.\r<br>여러분의 응원과 사랑으로 삼프로TV가 구독자...,삼프로TV_경제의신과함께,2022-06-30 13:54:12+00:00,0,-1 days +23:42:07,True
1,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,탁프로님 분석 명확하고 구체적이라서 도움 많이 받습니다,상상해보자,2022-07-07 02:52:59+00:00,0,6 days 12:40:54,False
2,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,김탁님 분석 최고.,자전거55,2022-07-04 08:00:00+00:00,0,3 days 17:47:55,False
3,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,산업구조가 일본에서 한국 그 다음 중국 쪽으로 흘러가는 것. 한국이 살 길은 기술...,꿈이야!,2022-06-30 23:18:01+00:00,0,0 days 09:05:56,False
4,2022-06-30 14:12:05+00:00,"중국과 LCD 치킨게임 ""버려야 산다""/제약·바이오 전환점 다가왔나?/누리호 보유국...",iHXeZ5v15dk,"누리호 얘기를 듣는데, 사회자분들의 코멘트가 너무 방해된다는 생각이 들정도로 매끄럽...",롱타이,2022-06-30 22:33:27+00:00,1,0 days 08:21:22,False
...,...,...,...,...,...,...,...,...,...
274493,2021-06-01 00:44:02+00:00,"국내 전기차 업계.. 다시 치고 올라가나?_오늘아침 page2_유지웅, 박병창, 염승환",Qn2zNAAn7qw,블랙핑크와 bts는 전세계 1위입니다,서프라이즈슈퍼,2021-06-01 00:53:22+00:00,0,0 days 00:09:20,False
274494,2021-06-01 00:44:02+00:00,"국내 전기차 업계.. 다시 치고 올라가나?_오늘아침 page2_유지웅, 박병창, 염승환",Qn2zNAAn7qw,잘나다니까. 그저 앵커나 모든 사람이 잘해서 잘나갈 수 밖에 없다고 알지요. 그것은...,조금성,2021-06-01 00:51:03+00:00,0,0 days 00:07:01,False
274495,2021-06-01 00:44:02+00:00,"국내 전기차 업계.. 다시 치고 올라가나?_오늘아침 page2_유지웅, 박병창, 염승환",Qn2zNAAn7qw,오늘도 좋은 내용 감사감사~,오리,2021-06-01 00:50:35+00:00,2,0 days 00:06:33,False
274496,2021-06-01 00:44:02+00:00,"국내 전기차 업계.. 다시 치고 올라가나?_오늘아침 page2_유지웅, 박병창, 염승환",Qn2zNAAn7qw,스팩관련주 너무 도박인가여?,Gu Park,2021-06-01 00:43:28+00:00,3,-1 days +23:59:26,False
